In [ ]:
import os
import random
import argparse

import pdb
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

from data_helper import LabeledDataset
from helper import compute_ats_bounding_boxes, compute_ts_road_map

from model_loader import get_transform_task1, get_transform_task2, ModelLoader

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)

verbose = True

image_folder = '/scratch/brs426/data'
annotation_csv = '/scratch/brs426/data/annotation.csv'

labeled_scene_index = np.arange(106, 109)

# For bounding boxes task
labeled_trainset_task1 = LabeledDataset(
    image_folder=image_folder,
    annotation_file=annotation_csv,
    scene_index=labeled_scene_index,
    transform=get_transform_task1(),
    extra_info=False
    )
dataloader_task1 = torch.utils.data.DataLoader(
    labeled_trainset_task1,
    batch_size=1,
    shuffle=False,
    num_workers=4
    )
# For road map task
labeled_trainset_task2 = LabeledDataset(
    image_folder=image_folder,
    annotation_file=annotation_csv,
    scene_index=labeled_scene_index,
    transform=get_transform_task2(),
    extra_info=False
    )
dataloader_task2 = torch.utils.data.DataLoader(
    labeled_trainset_task2,
    batch_size=1,
    shuffle=False,
    num_workers=4
    )

model_loader = ModelLoader()

total = 0
total_ats_bounding_boxes = 0
total_ts_road_map = 0
with torch.no_grad():
    for i, data in enumerate(dataloader_task1):
        total += 1
        sample, target, road_image = data
        sample = sample.cuda()

        predicted_bounding_boxes = model_loader.get_bounding_boxes(sample)[0].cpu()
        ats_bounding_boxes = compute_ats_bounding_boxes(predicted_bounding_boxes, target['bounding_box'][0])
        total_ats_bounding_boxes += ats_bounding_boxes

        if verbose:
            print(f'{i} - Bounding Box Score: {ats_bounding_boxes:.4}')

    for i, data in enumerate(dataloader_task2):
        sample, target, road_image = data
        sample = sample.cuda()

        predicted_road_map = model_loader.get_binary_road_map(sample).cpu()
        ts_road_map = compute_ts_road_map(predicted_road_map, road_image)
        total_ts_road_map += ts_road_map

        if verbose:
            print(f'{i} - Road Map Score: {ts_road_map:.4}')

print(f'{model_loader.team_name} - {model_loader.round_number} - Bounding Box Score: {total_ats_bounding_boxes / total:.4} - Road Map Score: {total_ts_road_map / total:.4}')
    











0 - Bounding Box Score: 0.009934
1 - Bounding Box Score: 0.04114
2 - Bounding Box Score: 0.03771
3 - Bounding Box Score: 0.04059
4 - Bounding Box Score: 0.05557
5 - Bounding Box Score: 0.04301
6 - Bounding Box Score: 0.02429
7 - Bounding Box Score: 0.02354
8 - Bounding Box Score: 0.04114
9 - Bounding Box Score: 0.04258
10 - Bounding Box Score: 0.04173
11 - Bounding Box Score: 0.02891
12 - Bounding Box Score: 0.02438
13 - Bounding Box Score: 0.0
14 - Bounding Box Score: 0.01166
15 - Bounding Box Score: 0.02146
16 - Bounding Box Score: 0.02891
17 - Bounding Box Score: 0.0
18 - Bounding Box Score: 0.04327
19 - Bounding Box Score: 0.009249
20 - Bounding Box Score: 0.01916
21 - Bounding Box Score: 0.02595
22 - Bounding Box Score: 0.02785
23 - Bounding Box Score: 0.01537
24 - Bounding Box Score: 0.04719
25 - Bounding Box Score: 0.0149
26 - Bounding Box Score: 0.04124
27 - Bounding Box Score: 0.0
28 - Bounding Box Score: 0.04883
29 - Bounding Box Score: 0.04254
30 - Bounding Box Score: 0.0268

In [2]:
def empty_func():
    xs = torch.Tensor([200, 210, 210, 200])
    ys = torch.Tensor([400, 400, 405, 405])

    xs = xs - 400
    ys = 800 - ys # right-side up
    ys = ys - 400

    xs /= 10.
    ys /= 10.

    coords = torch.stack((xs, ys))
    coords = [coords]

    coords = torch.stack(coords).double()
    return tuple([coords])